In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL-Advanced


In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    # 텍스트 데이터로부터 FAISS 벡터 저장소를 생성
    ["Jongjin is an AI engineer who loves programming!"],
    embedding=OpenAIEmbeddings(),
)

# 벡터 저장소를 기반으로 retriever를 생성
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4o-mini")

# chain을 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# 그래프 구성 확인


In [4]:
# 체인의 그래프에서 노드를 가져옵니다.
chain.get_graph().nodes

{'85e8455874d54a218ed2fac237b7adaf': Node(id='85e8455874d54a218ed2fac237b7adaf', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None),
 'fe9144cf551f4ba88be7ad29aab13960': Node(id='fe9144cf551f4ba88be7ad29aab13960', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None),
 '3d3fb04b4283424ca2438d39a3b97e5c': Node(id='3d3fb04b4283424ca2438d39a3b97e5c', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000208FA8D8D50>, search_kwargs={}), metadata=None),
 'bd0c8a426ec046e6864bf0b51c874308': Node(id='bd0c8a426ec046e6864bf0b51c874308', name='Passthrough', data=RunnablePassthrough(), metadata=None),
 '6095314b5b0f48399b546057b19d7c01': Node(id='6095314b5b0f48399b546057b19d7c01', name='ChatPromptTemplate', dat

In [5]:
# 체인의 그래프에서 엣지를 가져옵니다.
chain.get_graph().edges

[Edge(source='262c9305ca9d46c3b176580ba2f82adc', target='40c679d88f3542c5a625ec757da61949', data=None, conditional=False),
 Edge(source='40c679d88f3542c5a625ec757da61949', target='4adf658cbce04fd09d9a946a71bb288a', data=None, conditional=False),
 Edge(source='262c9305ca9d46c3b176580ba2f82adc', target='e1681480e64648119656f50564708e72', data=None, conditional=False),
 Edge(source='e1681480e64648119656f50564708e72', target='4adf658cbce04fd09d9a946a71bb288a', data=None, conditional=False),
 Edge(source='4adf658cbce04fd09d9a946a71bb288a', target='4d2e228b6ae240539f05a2405e888ff2', data=None, conditional=False),
 Edge(source='4d2e228b6ae240539f05a2405e888ff2', target='069f3d9f90a540d5b4496f0d0ebbff02', data=None, conditional=False),
 Edge(source='772c431e8de74b2884d5fbbb846c40b4', target='908273ea4adb46459546756e497f3367', data=None, conditional=False),
 Edge(source='069f3d9f90a540d5b4496f0d0ebbff02', target='772c431e8de74b2884d5fbbb846c40b4', data=None, conditional=False)]

# 그래프 출력

In [6]:
# 체인의 그래프를 ASCII 형식으로 출력합니다.
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

# 프롬프트 가져오기

In [7]:
chain.get_prompts()  # 체인에서 사용되는 프롬프트를 가져옵니다.

[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])]